<a href="https://colab.research.google.com/github/ymuto0302/RW2024/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 転移学習のサンプルコード
- モデル：ResNet50
- データセット：CIFAR-10

実装メモ：
- データ拡張は行っていない。RandomFlipHorizontal 等を入れてもよいかも。
- 動作チェックが目的のため，エポック数は少なめ。

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# デバイスの設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# データ前処理の定義
transform = transforms.Compose([
    transforms.ToTensor(),  # PIL Image をテンソルに変換
])

# CIFAR-10 データセットの読み込み
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 事前学習済みモデルの読み込み
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# モデルの最後の層を新しいタスクに合わせて変更
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # 10は新しいクラス数

# 微調整のための準備
# 全ての層のパラメータを凍結（勾配計算を無効化）
for param in model.parameters():
    param.requires_grad = False

# 新しく追加した fc 層のパラメータのみ勾配計算を有効化
for param in model.fc.parameters():
    param.requires_grad = True

# 損失関数と最適化アルゴリズムの定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# モデルを GPU へ移す
model = model.to(device)

# 学習ループ
num_epochs=5
for epoch in range(num_epochs):
    # 学習
    train_loss = 0.0
    train_total = 0
    model.train()
    for inputs, labels in train_loader:  # dataloader は別途定義が必要
        # GPU にデータを移す
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() # 勾配の初期化
        outputs = model(inputs) # 順伝播
        loss = criterion(outputs, labels) # 損失の計算

        # 損失のログ取り
        train_loss += loss.item()
        train_total += labels.size(0)

        loss.backward() # 逆伝搬
        optimizer.step() # パラメータ更新

    # 評価
    correct = 0
    total = 0
    model.eval()
    for inputs, labels in test_loader:  # test_dataloader は別途定義が必要
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # 現在のエポックの精度を表示
    print(f"Epoch {epoch+1}/{num_epochs},  Train Loss:{train_loss/train_total:.4f}, Test Accuracy: {correct / total:.4f}")

Using device: cuda:0
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/5,  Train Loss:0.0261, Accuracy: 0.4768
Epoch 2/5,  Train Loss:0.0237, Accuracy: 0.4839
Epoch 3/5,  Train Loss:0.0231, Accuracy: 0.4884


KeyboardInterrupt: 